In [30]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.widgets as wid
import scipy.signal as sig
import scipy.signal.windows as win
import scipy 
import scipy.io
from scipy.signal import lfilter

from numpy.random import default_rng
rng = default_rng()

import util

In [31]:
## 1. Étude Théorique : Analyse du mélange de deux processus

### Montrer que

$$
S_Z(\nu) = S_Y(\nu) + S_T(\nu)
$$

On a $Z = h \star X + Y$

$h$ est stable, $X$ est SSL donc $T$ est SSL par le théorème de filtrage des processus SSL. Par le résultat admis, on a $T$ et $Y$ indépendants.

On montre d'abord que $Z$ est SSL.

1. On a, pour tout $n$,
$$
E(|T_n + Y_n|^2) = E(|T_n|^2 + 2 |T_nY_n| + |Y_n|^2)
$$
$$
= E(|T_n|^2) + 2E(|T_nY_n|) + E(|Y_n|^2)
$$
Avec $T_n$ et $Y_n$ indépendants et SSL:
$$
= E(|T_n|^2) + E(|T_n|)E(|Y_n|) + E(|Y_n|^2) < +\infty
$$

2. On a bien :
$$
E(Z_n) = E(T_n) + E(Y_n) = m_T + m_Y = m_Z, \forall n
$$

3. On a :
$$
R_Z(k) = R_T(k) + R_Y(k) + E(T_{n+k}^{c}\bar{Y_{n}^c}) + E(\bar{T_{n}^c}Y_{n+k}^{c})
$$
Avec les termes de covariance nuls car $T$ et $Y$ sont indépendants :
$$
R_Z(k) = R_T(k) + R_Y(k)
$$
Ne dépend que de $k$, a fortiori :
$$
S_Z(\nu) = S_T(\nu) + S_Y(\nu)
$$

### Montrer que

Quelque soit la suite sommable $h^1$ et on notant $Z^1$ le processus $Z^1 = Z − h^1 ∗ X$ on a : 

$$
R_{Z^1}(0) \geq R_Y(0)
$$
Et si $R_{Z^1}(0) \geq R_Y(0)$ alors $h^1 = h$.


Par le calcul on a :
$$
R_{Z^1}(0) = R_Y(0) + R_{(h - h^1) ∗ X_n}(0) \geq 0
$$
Soit :
$$
R_{Z^1}(0) \geq R_Y(0)
$$
Et si $h^1 = h$ on obtient :
$$
R_{Z^1}(0) = R_Y(0) + R_{\tilde{0} ∗ X_n}(0) = R_Y(0)
$$

### 1.1 Première application : Identifier un canal

In [32]:
# Donnés
n = 1000
sigma = 0.1
h = np.array([1, 0.5])
# Calculés
x = rng.random(n)
y = sigma * rng.random(n)
z = lfilter(h, [1], x) + y

In [33]:
# Estimés
p = 5

h_estimated = util.moindres_carres(p, x, z)
h_err = util.norm(h_estimated - h) / util.norm(h)

plt.figure(figsize=(8, 6))
plt.subplot(3, 1, 1)
plt.title("h^1 estimé")
plt.plot(h_estimated)
plt.subplot(3, 1, 2)
plt.title("h réel")
plt.plot(h)
plt.subplot(3, 1, 3)
plt.title("Erreur")
plt.plot(h_err)

AttributeError: module 'util' has no attribute 'norm'

On déterminera la longueur de $h$ empiriquement.

Si $h$ trop court, ça ne va pas fonctionner, si
$h$ trop long les derniers coefficients seront nuls et on pourra réduire.

## 2. Le modèle auto-régressif pour la synthèse de parole

$\epsilon_n$ est tel que :
$$
X_n + h_1X_{n-1} + ... + h_p X_{n-p} = \epsilon_n
$$